#**Predicting Mortgage Backed Securities Prepayment Risk Prediction**

#**Importing Data & Needed Libraries**

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import networkx as nx
import statsmodels.api as sm
from sklearn import preprocessing
from sklearn.preprocessing import OneHotEncoder
import warnings
warnings.filterwarnings("ignore")
import matplotlib
warnings.filterwarnings("always")

%matplotlib inline

In [ ]:
data = pd.read_csv('LoanExport.csv')
data.head(20)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
<frozen importlib._bootstrap>:914: ImportWarning: APICoreClientInfoImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:914: ImportWarning: _PyDriveImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:914: ImportWarning: _OpenCVImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:914: ImportWarning: _BokehImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:914: ImportWarning: _AltairImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:

,CreditScore,FirstPaymentDate,FirstTimeHomebuyer,MaturityDate,MSA,MIP,Units,Occupancy,OCLTV,DTI,...,PostalCode,LoanSeqNum,LoanPurpose,OrigLoanTerm,NumBorrowers,SellerName,ServicerName,EverDelinquent,MonthsDelinquent,MonthsInRepayment
0,0,199902,N,202901,16974,25,1,O,89,27,...,60400,F199Q1268030,P,360,2,FL,WASHINGTONMUTUALBANK,0.0,0.0,52.0
1,0,199902,N,202901,19740,0,1,O,73,17,...,80200,F199Q1015092,N,360,1,FT,CHASEHOMEFINANCELLC,0.0,0.0,144.0
2,0,199902,N,202901,29940,0,1,O,75,16,...,66000,F199Q1266886,N,360,2,FL,WASHINGTONMUTUALBANK,0.0,0.0,67.0
3,0,199902,N,202901,31084,0,1,O,76,14,...,90700,F199Q1178167,N,360,2,GM,GMACMTGECORP,0.0,0.0,35.0
4,0,199902,N,202901,35644,0,1,O,78,18,...,7600,F199Q1178517,N,360,2,GM,GMACMTGECORP,0.0,0.0,54.0
5,0,199902,N,202901,X,25,1,O,89,40,...,80400,F199Q1224802,N,360,2,Ot,Other servicers,0.0,0.0,42.0
6,0,199902,N,202901,X,25,1,O,90,21,...,53100,F199Q1291436,P,360,1,RE,Other servicers,0.0,0.0,44.0
7,0,199902,X,202901,36740,0,1,O,72,20,...,32800,F199Q1001824,N,360,1,NO,WELLSFARGOBANKNA,1.0,3.0,145.0
8,0,199902,Y,202710,25540,30,1,O,95,38,...,6400,F199Q1079744,P,345,1,Ot,Other servicers,0.0,0.0,64.0
9,0,199902,Y,202901,48620,30,1,O,95,27,...,67000,F199Q1196318,P,360,1,Ot,WELLSFARGOBANKNA,1.0,104.0,212.0


**Data Preprocessing & EDA**

In [ ]:
data.shape

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


(8288, 28)

In [ ]:
# Checking informations about our features
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8288 entries, 0 to 8287
Data columns (total 28 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   CreditScore         8288 non-null   int64  
 1   FirstPaymentDate    8288 non-null   int64  
 2   FirstTimeHomebuyer  8288 non-null   object 
 3   MaturityDate        8288 non-null   int64  
 4   MSA                 8288 non-null   object 
 5   MIP                 8288 non-null   int64  
 6   Units               8288 non-null   int64  
 7   Occupancy           8288 non-null   object 
 8   OCLTV               8288 non-null   int64  
 9   DTI                 8288 non-null   int64  
 10  OrigUPB             8288 non-null   int64  
 11  LTV                 8288 non-null   int64  
 12  OrigInterestRate    8288 non-null   float64
 13  Channel             8288 non-null   object 
 14  PPM                 8288 non-null   object 
 15  ProductType         8288 non-null   object 
 16  Proper

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
# Checking for missing values
data.isnull().sum()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


CreditScore             0
FirstPaymentDate        0
FirstTimeHomebuyer      0
MaturityDate            0
MSA                     0
MIP                     0
Units                   0
Occupancy               0
OCLTV                   0
DTI                     0
OrigUPB                 0
LTV                     0
OrigInterestRate        0
Channel                 0
PPM                     0
ProductType             0
PropertyState           0
PropertyType            0
PostalCode              0
LoanSeqNum              0
LoanPurpose             0
OrigLoanTerm            0
NumBorrowers            0
SellerName            782
ServicerName            1
EverDelinquent          1
MonthsDelinquent        1
MonthsInRepayment       1
dtype: int64

In [ ]:
#checking for duplicates values to drop them
duplicate=data.duplicated()
print(duplicate.sum())

0


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
#Check if there's any categorical features in our data
data.dtypes

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


CreditScore             int64
FirstPaymentDate        int64
FirstTimeHomebuyer     object
MaturityDate            int64
MSA                    object
MIP                     int64
Units                   int64
Occupancy              object
OCLTV                   int64
DTI                     int64
OrigUPB                 int64
LTV                     int64
OrigInterestRate      float64
Channel                object
PPM                    object
ProductType            object
PropertyState          object
PropertyType           object
PostalCode              int64
LoanSeqNum             object
LoanPurpose            object
OrigLoanTerm            int64
NumBorrowers           object
SellerName             object
ServicerName           object
EverDelinquent        float64
MonthsDelinquent      float64
MonthsInRepayment     float64
dtype: object

In [ ]:
# Convert date feature to datetime format
data['MaturityDate'] = data['MaturityDate'].apply(lambda x: pd.to_datetime(str(x), format='%Y%m'))

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
data['FirstPaymentDate'] = data['FirstPaymentDate'].apply(lambda x: pd.to_datetime(str(x), format='%Y%m'))

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
#Visualize the change on dtypes of features
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8288 entries, 0 to 8287
Data columns (total 28 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   CreditScore         8288 non-null   int64         
 1   FirstPaymentDate    8288 non-null   datetime64[ns]
 2   FirstTimeHomebuyer  8288 non-null   object        
 3   MaturityDate        8288 non-null   datetime64[ns]
 4   MSA                 8288 non-null   object        
 5   MIP                 8288 non-null   int64         
 6   Units               8288 non-null   int64         
 7   Occupancy           8288 non-null   object        
 8   OCLTV               8288 non-null   int64         
 9   DTI                 8288 non-null   int64         
 10  OrigUPB             8288 non-null   int64         
 11  LTV                 8288 non-null   int64         
 12  OrigInterestRate    8288 non-null   float64       
 13  Channel             8288 non-null   object      

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
#View description of all of our features
num1_cols = data.describe(include='all')
num1_cols

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
<ipython-input-10-73e8b8f9509f>:2: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  num1_cols = data.describe(include='all')
<ipython-input-10-73e8b8f9509f>:2: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  num1_cols = data.describe(include='a

,CreditScore,FirstPaymentDate,FirstTimeHomebuyer,MaturityDate,MSA,MIP,Units,Occupancy,OCLTV,DTI,...,PostalCode,LoanSeqNum,LoanPurpose,OrigLoanTerm,NumBorrowers,SellerName,ServicerName,EverDelinquent,MonthsDelinquent,MonthsInRepayment
count,8288.000000,8288,8288,8288,8288,8288.000000,8288.000000,8288,8288.000000,8288.000000,...,8288.000000,8288,8288,8288.000000,8288,7506,8287,8287.000000,8287.000000,8287.000000
unique,NaN,31,3,56,354,NaN,NaN,3,NaN,NaN,...,NaN,8288,3,NaN,3,20,20,NaN,NaN,NaN
top,NaN,1999-05-01 00:00:00,N,2029-04-01 00:00:00,X,NaN,NaN,O,NaN,NaN,...,NaN,F199Q1268030,P,NaN,2,Ot,Other servicers,NaN,NaN,NaN
freq,NaN,3108,4722,3103,1194,NaN,NaN,7982,NaN,NaN,...,NaN,1,3578,NaN,5219,2170,2187,NaN,NaN,NaN
first,NaN,1999-02-01 00:00:00,NaN,2024-09-01 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
last,NaN,2003-04-01 00:00:00,NaN,2030-10-01 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mean,497.931226,NaN,NaN,NaN,NaN,12.749035,1.029199,NaN,80.860763,29.553571,...,53255.091699,NaN,NaN,359.702461,NaN,NaN,NaN,0.440087,6.433088,63.205623
std,205.132493,NaN,NaN,NaN,NaN,13.259903,0.208117,NaN,13.005811,13.156933,...,30311.163141,NaN,NaN,3.006374,NaN,NaN,NaN,0.496427,16.473469,50.595431
min,0.000000,NaN,NaN,NaN,NaN,0.000000,1.000000,NaN,7.000000,0.000000,...,600.000000,NaN,NaN,306.000000,NaN,NaN,NaN,0.000000,0.000000,1.000000
25%,552.000000,NaN,NaN,NaN,NaN,0.000000,1.000000,NaN,75.000000,22.000000,...,29500.000000,NaN,NaN,360.000000,NaN,NaN,NaN,0.000000,0.000000,33.000000


In [ ]:
#Count number of distinct elements in axis 0.
data.nunique()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


CreditScore            142
FirstPaymentDate        31
FirstTimeHomebuyer       3
MaturityDate            56
MSA                    354
MIP                     18
Units                    4
Occupancy                3
OCLTV                   83
DTI                     66
OrigUPB                270
LTV                     82
OrigInterestRate        66
Channel                  4
PPM                      3
ProductType              1
PropertyState           53
PropertyType             6
PostalCode             743
LoanSeqNum            8288
LoanPurpose              3
OrigLoanTerm            44
NumBorrowers             3
SellerName              20
ServicerName            20
EverDelinquent           2
MonthsDelinquent       134
MonthsInRepayment      212
dtype: int64

In [ ]:
data["EverDelinquent"].value_counts()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


0.0    4640
1.0    3647
Name: EverDelinquent, dtype: int64

In [ ]:
print("Seller Name size: \n",data.SellerName.value_counts(),"\n\n")

Seller Name size: 
 Ot    2170
NO     866
ST     633
FI     627
FL     474
GM     346
CO     327
BA     294
WA     292
FT     249
BI     202
PR     170
OL     167
CH     165
PN     141
AC     120
HO     110
CR      78
G       42
RE      33
Name: SellerName, dtype: int64 




/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
#Return unique values based on a hash table.
data['FirstTimeHomebuyer'].unique()
# X means No
# Y means Yes
# N means Unknown value

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


array(['N', 'X', 'Y'], dtype=object)

In [ ]:
data['PPM'].unique()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


array(['N', 'X', 'Y'], dtype=object)

In [ ]:
data['NumBorrowers'].unique()
# X  means unknown value

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


array(['2', '1', 'X '], dtype=object)

#**DATA ENCODING**

Encoding the categorical columns

1 - Label Encoder - If you have 2 categories which may or may not be not in order then you use label encoder

2 - one hot Encoder - more then number of categories you use one hot encoding which means that you are converting a value in the row into a column

In [ ]:
# Define the columns to be label encoded and one-hot encoded
label_encode_columns = ['CreditScore', 'LoanPurpose']  # Replace with your column names
one_hot_encode_columns = ['Occupancy', 'SellerName']  # Replace with your column names


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
for column in label_encode_columns:
    data[column] = label_encoder.fit_transform(data[column])

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
# Apply One-Hot Encoding to selected columns
one_hot_encoder = OneHotEncoder(drop='first', sparse=False)
encoded_data = one_hot_encoder.fit_transform(data[one_hot_encode_columns])

# Get the column names for the one-hot encoded columns
encoded_column_names = one_hot_encoder.get_feature_names_out(input_features=one_hot_encode_columns)

# Create a DataFrame with the encoded data and column names
encoded_data = pd.DataFrame(encoded_data, columns=encoded_column_names)

# Combine the encoded data with the original DataFrame
data = pd.concat([data, encoded_data], axis=1)

# Drop the original columns that were one-hot encoded
data.drop(one_hot_encode_columns, axis=1, inplace=True)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


#**DATA LABELLING**

In [ ]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
# Assuming you have a column named 'CreditScore' containing non-numeric labels
# You can use LabelEncoder to encode it
label_encoder = LabelEncoder()
data['CreditScore'] = label_encoder.fit_transform(data['CreditScore'])

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
